<a href="https://colab.research.google.com/github/victorialovefranklin/Toward-Climate-Resilient-Energy-Systems/blob/main/EAGLE_I_Aggregated_(Statewide)_Trends_Models_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part I: Comparative Validation of EAGLE-I™ Outage Records Against EIA-861 Sales/Service Data with DOE-417 Event Grid Disturbances (California, 2014–2023)**

Code ngests an EAGLE-I aggregated CSV, builds monthly/yearly trend tables, exports CSV + Excel, generates matplotlib visuals (no seaborn), performs a model comparison (XGBoost, Random Forest, KNN, SVM, MLP) with metrics (RMSE/MAE/R²/MAPE), makes a 12-month forecast with the best model, and writes an AI-style summary.

In [21]:
!pip install pandas numpy matplotlib scikit-learn xgboost

In [22]:
!pip install pandas numpy matplotlib scikit-learn

In [26]:
display(metrics_df)
print("\nBest Model Performance:")
display(best_row)

,model,RMSE,MAE,R2,MAPE_%
2,SVM,8.283963e+06,4.537824e+06,-0.419102,38.903934
3,MLP,9.762009e+06,6.700981e+06,-0.970679,77.809226
4,XGBoost,1.101611e+07,7.366083e+06,-1.509537,118.184230
0,RandomForest,1.116944e+07,9.031316e+06,-1.579884,151.002793
1,KNN,1.152740e+07,8.367808e+06,-1.747894,145.420107



Best Model Performance:


,2
model,SVM
RMSE,8283962.672222
MAE,4537824.122213
R2,-0.419102
MAPE_%,38.903934


In [52]:
# =============================================================
# NOAA Storm Events (California, 2010–2024) – All Event Types
# =============================================================

import os, re, requests, gzip
import pandas as pd
from io import BytesIO
from tqdm import tqdm

# --- CONFIG ---
BASE = "https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/"
YEARS = range(2010, 2025)
OUTDIR = "/content/stormevents_raw"
os.makedirs(OUTDIR, exist_ok=True)

# --- Discover available yearly files ---
html = requests.get(BASE).text
pattern = re.compile(r"StormEvents_details-ftp_v1\.0_d(19|20)\d{2}_c\d+\.csv\.gz")
files = sorted(set(m.group(0) for m in pattern.finditer(html)))
def year_from_name(fn): return int(re.search(r"d(\d{4})_", fn).group(1))
files = [fn for fn in files if year_from_name(fn) in YEARS]
print(f"Found {len(files)} NOAA detail files.")

# --- Helper: convert '5K','2.3M' strings to USD numeric ---
def parse_damage(s):
    if pd.isna(s): return 0.0
    s = str(s).strip().upper()
    if not s or s == "0": return 0.0
    mult = 1
    if s.endswith("K"): mult, s = 1e3, s[:-1]
    elif s.endswith("M"): mult, s = 1e6, s[:-1]
    elif s.endswith("B"): mult, s = 1e9, s[:-1]
    try:
        return float(s) * mult
    except:
        return 0.0

# --- Download + Filter + Combine ---
dfs = []
for fn in tqdm(files, desc="Downloading"):
    url = BASE + fn
    r = requests.get(url)
    r.raise_for_status()
    with gzip.open(BytesIO(r.content), "rt", encoding="latin1") as f:
        df = pd.read_csv(f, low_memory=False)
    # California only
    df = df[df["STATE"].str.upper() == "CALIFORNIA"]
    if df.empty: continue
    # Keep counties (CZ_TYPE = 'C')
    df = df[df["CZ_TYPE"] == "C"]
    df["county"] = df["CZ_NAME"].str.title().str.strip()
    # Monthly date
    df["BEGIN_YEARMONTH"] = df["BEGIN_YEARMONTH"].astype(str)
    df["month"] = pd.to_datetime(df["BEGIN_YEARMONTH"] + "01", format="%Y%m%d")
    # Numeric damages
    for col in ["DAMAGE_PROPERTY", "DAMAGE_CROPS"]:
        df[col + "_USD"] = df[col].map(parse_damage)
    dfs.append(df[
        ["county","month","EVENT_TYPE","EVENT_ID",
         "DAMAGE_PROPERTY_USD","DAMAGE_CROPS_USD"]
    ])

events = pd.concat(dfs, ignore_index=True)
print("✅ Total California events loaded:", len(events))

# --- Aggregate by County + Month + Event Type ---
county_month_type = (
    events.groupby(["county","month","EVENT_TYPE"], as_index=False)
    .agg(
        events_n=("EVENT_ID","count"),
        prop_damage_usd=("DAMAGE_PROPERTY_USD","sum"),
        crop_damage_usd=("DAMAGE_CROPS_USD","sum")
    )
)

# --- Summary of total counts per Event Type (all years) ---
event_summary = (
    events.groupby("EVENT_TYPE", as_index=False)
    .agg(total_events=("EVENT_ID","count"))
    .sort_values("total_events", ascending=False)
)

# --- Save both outputs ---
path_detail = "/content/ca_stormevents_county_month_eventtype_2010_2024.csv"
path_summary = "/content/ca_stormevents_eventtype_summary_2010_2024.csv"
county_month_type.to_csv(path_detail, index=False)
event_summary.to_csv(path_summary, index=False)

print(f"\nDetailed file: {path_detail}")
print(f"Summary file:  {path_summary}\n")
print(event_summary.head(15))



Found 15 NOAA detail files.


Downloading: 100%|██████████| 15/15 [00:40<00:00,  2.72s/it]

✅ Total California events loaded: 8216

Detailed file: /content/ca_stormevents_county_month_eventtype_2010_2024.csv
Summary file:  /content/ca_stormevents_eventtype_summary_2010_2024.csv

           EVENT_TYPE  total_events
3               Flood          3086
2         Flash Flood          1733
6          Heavy Rain          1305
0         Debris Flow           633
9   Thunderstorm Wind           586
5                Hail           426
4        Funnel Cloud           175
8           Lightning           149
10            Tornado           113
1          Dust Devil             8
7           High Wind             1
11           Wildfire             1


In [56]:
# ================================================
# EAGLE-I Aggregated (Statewide) — Trends, Models, Forecast (Adaptive)
# ================================================

import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    HAS_XGB = False

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import clone

# --------------------
# 0) CONFIG
# --------------------
CANDIDATE_INPUTS = [
    "/content/Eaglei_CA_Outages_Agg.csv",
    "/mnt/data/Eaglei_CA_Outages_Agg.csv",
    "./Eaglei_CA_Outages_Agg.csv",
]
INPUT_CSV = next((p for p in CANDIDATE_INPUTS if Path(p).exists()), CANDIDATE_INPUTS[0])

OUT_DIR = Path("/content/eaglei_agg_outputs_state") if Path("/content").exists() else Path("./eaglei_agg_outputs_state")
OUT_DIR.mkdir(parents=True, exist_ok=True)

TARGET_COL = "customer_weighted_hours"  # or "outage_count"

# --------------------
# 1) LOAD
# --------------------
if not Path(INPUT_CSV).exists():
    raise FileNotFoundError(f"File not found: {INPUT_CSV}")

df_src = pd.read_csv(INPUT_CSV, low_memory=False)
print(f"✅ Loaded: {INPUT_CSV}")
print("Columns:", df_src.columns.tolist())

col_map = {}
for want in ["state", "year", "month", "outage_count", "max_outage_duration", "customer_weighted_hours"]:
    for c in df_src.columns:
        if c.lower() == want:
            col_map[want] = c
            break

for need in ["year", "month", TARGET_COL]:
    if need not in col_map:
        raise ValueError(f"Required column missing in file for '{need}'. Detected map: {col_map}")

# --------------------
# 2) CLEAN & MONTH
# --------------------
df_src[col_map["year"]] = pd.to_numeric(df_src[col_map["year"]], errors="coerce").astype("Int64")
df_src[col_map["month"]] = pd.to_numeric(df_src[col_map["month"]], errors="coerce").astype("Int64")
df_src = df_src.dropna(subset=[col_map["year"], col_map["month"]]).copy()

mn = df_src[col_map["month"]].dropna().astype(int)
if len(mn) and mn.min() == 0 and mn.max() == 11:
    df_src.loc[:, col_map["month"]] = df_src[col_map["month"]] + 1

valid_mask = (df_src[col_map["month"]] >= 1) & (df_src[col_map["month"]] <= 12)
dropped = int((~valid_mask).sum())
if dropped > 0:
    print(f"⚠️ Dropping {dropped} rows with invalid month outside 1..12.")
df_src = df_src.loc[valid_mask].copy()

df_src["MONTH"] = pd.to_datetime(
    {"year": df_src[col_map["year"]].astype(int),
     "month": df_src[col_map["month"]].astype(int),
     "day": 1}
)

df_src[col_map[TARGET_COL]] = pd.to_numeric(df_src[col_map[TARGET_COL]], errors="coerce")
df_src = df_src.dropna(subset=[col_map[TARGET_COL]]).copy()

# --------------------
# 3) AGG TABLES & PLOTS
# --------------------
state_month = (
    df_src.groupby("MONTH", as_index=False)[[col_map[TARGET_COL]]]
          .sum()
          .rename(columns={col_map[TARGET_COL]: "y"})
          .sort_values("MONTH").reset_index(drop=True)
)
state_month["YEAR"] = state_month["MONTH"].dt.year
state_year = state_month.groupby("YEAR", as_index=False)[["y"]].sum()

# export tables
state_month.to_csv(OUT_DIR / "state_month.csv", index=False)
state_year.to_csv(OUT_DIR / "state_year.csv", index=False)
with pd.ExcelWriter(OUT_DIR / "eaglei_state_trends.xlsx") as xw:
    state_month.to_excel(xw, sheet_name="state_month", index=False)
    state_year.to_excel(xw, sheet_name="state_year", index=False)

print(f"✅ Exported summary tables to {OUT_DIR}")

# plots
plt.figure()
plt.plot(state_month["MONTH"], state_month["y"], marker="o")
plt.title("Statewide Monthly — Target"); plt.xlabel("Month"); plt.ylabel(TARGET_COL.replace("_"," ").title())
plt.tight_layout(); plt.savefig(OUT_DIR / "fig_state_month.png", dpi=220); plt.close()

plt.figure()
plt.plot(state_year["YEAR"], state_year["y"], marker="o")
plt.title("Statewide Yearly — Target"); plt.xlabel("Year"); plt.ylabel(TARGET_COL.replace("_"," ").title())
plt.tight_layout(); plt.savefig(OUT_DIR / "fig_state_year.png", dpi=220); plt.close()

# ------------------------------------------------
# 4) EXOGENOUS (OPTIONAL) — helper funcs + loaders
# ------------------------------------------------
def _safe_read_csv(path, **kw):
    encodings = ["utf-8", "utf-8-sig", "latin1", "cp1252"]
    last_err = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False, **kw)
        except Exception as e:
            last_err = e
    try:
        return pd.read_csv(path, encoding="utf-8", on_bad_lines="skip",
                           encoding_errors="replace", low_memory=False, **kw)
    except Exception as e:
        print(f"⚠️ Could not read {path}: {last_err or e}")
        return None

def _to_month(series):
    s = pd.to_datetime(series, errors="coerce")
    try:
        if getattr(s.dt, "tz", None) is not None:
            try: s = s.dt.tz_convert(None)
            except Exception: s = s.dt.tz_localize(None)
    except Exception:
        pass
    return s.dt.to_period("M").dt.to_timestamp()

def _coerce_month(df, ycol, mcol):
    if ycol not in df.columns or mcol not in df.columns:
        return None
    tmp = df.copy()
    tmp[ycol] = pd.to_numeric(tmp[ycol], errors="coerce").astype("Int64")
    tmp[mcol] = pd.to_numeric(tmp[mcol], errors="coerce").astype("Int64")
    tmp = tmp.dropna(subset=[ycol, mcol]).copy()
    if tmp[mcol].min() == 0 and tmp[mcol].max() == 11:
        tmp[mcol] = tmp[mcol] + 1
    tmp = tmp[(tmp[mcol] >= 1) & (tmp[mcol] <= 12)].copy()
    tmp["MONTH"] = pd.to_datetime({"year": tmp[ycol].astype(int), "month": tmp[mcol].astype(int), "day": 1})
    return tmp

def _collapse_suffix_pairs(df):
    cols = list(df.columns)
    for c in cols:
        if c.endswith("_x"):
            base = c[:-2]; cy = base + "_y"
            if cy in df.columns:
                if pd.api.types.is_numeric_dtype(df[c]) and pd.api.types.is_numeric_dtype(df[cy]):
                    df[base] = df[c].fillna(0) + df[cy].fillna(0)
                else:
                    df[base] = df[c].combine_first(df[cy])
                df.drop([c, cy], axis=1, inplace=True)
    return df

def _first_existing(paths):
    for p in paths:
        if Path(p).exists(): return p
    return None

exo_parts, added_feats = [], []

# (A) NOAA Storm Events
p_storm = _first_existing([
    "/content/ca_stormevents_county_month_eventtype_2010_2024.csv",
    "/content/ca_stormevents_county_month_2010_2024.csv",
    "/mnt/data/ca_stormevents_county_month_eventtype_2010_2024.csv",
    "/mnt/data/ca_stormevents_county_month_2010_2024.csv",
])
if p_storm:
    st = _safe_read_csv(p_storm)
    if st is not None:
        if "MONTH" in st.columns: st["MONTH"] = _to_month(st["MONTH"])
        elif "month" in st.columns: st["MONTH"] = _to_month(st["month"])
        else:
            for c in st.columns:
                if "date" in c.lower() or "month" in c.lower():
                    st["MONTH"] = _to_month(st[c]); break
        sums = []
        if "events_n" in st.columns:        sums.append(("storm_events_n", ("events_n","sum")))
        if "prop_damage_usd" in st.columns: sums.append(("storm_prop_damage_usd", ("prop_damage_usd","sum")))
        if "crop_damage_usd" in st.columns: sums.append(("storm_crop_damage_usd", ("crop_damage_usd","sum")))
        if "MONTH" in st.columns and sums:
            exo_storm = st.groupby("MONTH", as_index=False).agg({old:how for (new,(old,how)) in sums})
            for new,(old,_) in sums:
                if old in exo_storm.columns:
                    exo_storm = exo_storm.rename(columns={old:new}); added_feats.append(new)
            exo_parts.append(exo_storm)

# (B) PSPS
p_psps = _first_existing(["/mnt/data/PSPS Event Rollup.csv","/content/PSPS Event Rollup.csv"])
if p_psps:
    psps = _safe_read_csv(p_psps)
    if psps is not None:
        date_col = None
        for c in psps.columns:
            if c.lower() in ("startdate","start_date","date","month"): date_col = c; break
        if date_col is None:
            tmp = _coerce_month(psps, "year", "month")
            if tmp is not None: psps = tmp
        if psps is not None:
            if "MONTH" not in psps.columns and date_col is not None:
                psps["MONTH"] = _to_month(psps[date_col])
            agg_map = {}
            for cand in ["Events","events","psps_events","EventCount"]:
                if cand in psps.columns: agg_map[cand] = "sum"
            for cand in ["CustomersImpacted","customers_impacted","Customers","customers_out"]:
                if cand in psps.columns: agg_map[cand] = "sum"
            if agg_map:
                exo_psps = psps.groupby("MONTH", as_index=False).agg(agg_map)
                rename_map = {}
                for c in list(agg_map.keys()):
                    if c.lower() in ("events","eventcount","psps_events"): rename_map[c] = "psps_events"
                    elif "customer" in c.lower(): rename_map[c] = "psps_customers_impacted"
                exo_psps = exo_psps.rename(columns=rename_map)
                added_feats += [v for v in rename_map.values()]
                exo_parts.append(exo_psps)

# (C) Planned/Unplanned
p_out = _first_existing(["/mnt/data/Planned_Unplanned_Outages.csv","/content/Planned_Unplanned_Outages.csv"])
if p_out:
    outg = _safe_read_csv(p_out)
    if outg is not None:
        if "MONTH" not in outg.columns:
            for c in outg.columns:
                if "date" in c.lower() or "month" in c.lower():
                    outg["MONTH"] = _to_month(outg[c]); break
        agg_map = {}
        for nm in outg.columns:
            l = nm.lower()
            if "customer" in l and ("out" in l or "impact" in l or "affect" in l): agg_map[nm] = "sum"
            if "outage" in l and ("count" in l or "events" in l): agg_map[nm] = "sum"
            if "minutes" in l or "duration" in l: agg_map[nm] = "sum"
        if "MONTH" in outg.columns and agg_map:
            exo_out = outg.groupby("MONTH", as_index=False).agg(agg_map)
            ren = {}
            for c in exo_out.columns:
                lc = c.lower()
                if "customer" in lc and ("out" in lc or "impact" in lc or "affect" in lc): ren[c] = "out_customers_impacted_sum"
                if "outage" in lc and ("count" in lc or "events" in lc): ren[c] = "out_events_n"
                if "minutes" in lc or "duration" in lc: ren[c] = "out_minutes_sum"
            exo_out = exo_out.rename(columns=ren)
            added_feats += [v for v in ren.values() if v != "MONTH"]
            exo_parts.append(exo_out)

# (D) Wildfires
p_wf = _first_existing(["/mnt/data/WildFires_Incident_Locations.csv","/content/WildFires_Incident_Locations.csv"])
if p_wf:
    wf = _safe_read_csv(p_wf)
    if wf is not None:
        date_col = None
        for c in wf.columns:
            if any(k in c.lower() for k in ["start","date","discovery","ig_date","incidentdate"]):
                date_col = c; break
        if date_col is None:
            for c in wf.columns:
                if "date" in c.lower(): date_col = c; break
        if date_col is not None:
            wf["MONTH"] = _to_month(wf[date_col])
            wf_m = wf.groupby("MONTH", as_index=False).agg(wildfire_incidents_n=("MONTH","size"))
            if "IncidentAcres" in wf.columns:
                acres = wf.groupby("MONTH", as_index=False)["IncidentAcres"].sum()
                wf_m = wf_m.merge(acres.rename(columns={"IncidentAcres":"wildfire_acres_burned"}), on="MONTH", how="left")
            added_feats += [c for c in ["wildfire_incidents_n","wildfire_acres_burned"] if c in wf_m.columns]
            exo_parts.append(wf_m)

# (E) Weather/Fire summary
p_weather = _first_existing([
    "/mnt/data/CA_Weather_Fire_Dataset_1984-2025.csv",
    "/content/CA_Weather_Fire_Dataset_1984-2025.csv",
    "/content/CA_Weather_Fire_Dataset_1984-2025 (2).csv",
])
if p_weather:
    wx = _safe_read_csv(p_weather)
    if wx is not None:
        tmp = None
        lower = {c.lower() for c in wx.columns}
        if {"year","month"}.issubset(lower):
            ycol = next(c for c in wx.columns if c.lower()=="year")
            mcol = next(c for c in wx.columns if c.lower()=="month")
            tmp = _coerce_month(wx, ycol, mcol)
        else:
            for c in wx.columns:
                if "date" in c.lower() or "month" in c.lower():
                    tmp = wx.copy(); tmp["MONTH"] = _to_month(tmp[c]); break
        if tmp is not None:
            agg_map, ren = {}, {}
            for cand in ["precip","pcpn_total","prcp","Precipitation","Prcp_mm","Prcp_in"]:
                if cand in tmp.columns: agg_map[cand] = "sum"; ren[cand] = "pcpn_total"
            for cand in ["tavg","tmean","tavg_mean","tmax_mean","tmin_mean","avg_temp_f","avg_temp_c"]:
                if cand in tmp.columns: agg_map[cand] = "mean"; ren[cand] = "tavg_mean"
            for cand in ["snow","snow_total","snowfall"]:
                if cand in tmp.columns: agg_map[cand] = "mean"; ren[cand] = "snow_total"
            for cand in ["snwd","snow_depth","snowdepth"]:
                if cand in tmp.columns: agg_map[cand] = "mean"; ren[cand] = "snwd_mean"
            for cand in ["acres_burned","area_burned","wildfire_acres_burned"]:
                if cand in tmp.columns: agg_map[cand] = "sum"; ren[cand] = "wildfire_acres_burned"
            if agg_map:
                exo_wx = tmp.groupby("MONTH", as_index=False).agg(agg_map).rename(columns=ren)
                added_feats += [v for v in ren.values() if v != "MONTH"]
                exo_parts.append(exo_wx)

# (F) Real-time incidents
p_rti = _first_existing(["/mnt/data/Real_Time_Incidents.csv","/content/Real_Time_Incidents.csv"])
if p_rti:
    rti = _safe_read_csv(p_rti)
    if rti is not None:
        date_col = None
        for c in rti.columns:
            if "date" in c.lower() or "start" in c.lower() or "time" in c.lower(): date_col = c; break
        if date_col is not None:
            rti["MONTH"] = _to_month(rti[date_col])
            exo_rti = rti.groupby("MONTH", as_index=False).agg(incidents_n=("MONTH","size"))
            added_feats.append("incidents_n")
            exo_parts.append(exo_rti)

# (G) DOE-417 (optional)
p_doe = _first_existing(["/mnt/data/DOE_417.csv","/content/DOE_417.csv","/mnt/data/DOE-417.csv","/content/DOE-417.csv"])
if p_doe:
    d417 = _safe_read_csv(p_doe)
    if d417 is not None:
        date_col = None
        for c in d417.columns:
            if any(k in c.lower() for k in ["date","start","begin"]): date_col = c; break
        if date_col is not None:
            d417["MONTH"] = _to_month(d417[date_col])
            agg = {"doe417_events_n": ("MONTH","size")}
            for cand in d417.columns:
                cl = cand.lower()
                if "customer" in cl and ("affect" in cl or "out" in cl or "impact" in cl):
                    agg["doe_customers_affected_sum"] = (cand,"sum")
                if "duration" in cl or "minutes" in cl or "hours" in cl:
                    agg["doe_event_minutes_sum"] = (cand,"sum")
            exo_doe = d417.groupby("MONTH", as_index=False).agg(**agg)
            added_feats += [c for c in exo_doe.columns if c != "MONTH"]
            exo_parts.append(exo_doe)

# merge exogenous
if exo_parts:
    exo = exo_parts[0]
    for part in exo_parts[1:]:
        exo = exo.merge(part, on="MONTH", how="outer")
    exo = _collapse_suffix_pairs(exo).sort_values("MONTH").reset_index(drop=True)
    state_month = _collapse_suffix_pairs(state_month.merge(exo, on="MONTH", how="left"))
    state_month.to_csv(OUT_DIR / "state_month_with_exogenous.csv", index=False)
    print("✅ Added exogenous features:", ", ".join(sorted(set(added_feats))))
else:
    print("ℹ️ No external features added (no files found or recognized).")

# --------------------
# 5) FEATURES (ADAPTIVE) + MODELING
# --------------------
def mape(y_true, y_pred):
    yt = np.asarray(y_true, dtype=float)
    yp = np.asarray(y_pred, dtype=float)
    mask = yt != 0
    return float(np.mean(np.abs((yt[mask] - yp[mask]) / yt[mask])) * 100.0) if mask.any() else np.nan

dfm = state_month[["MONTH","y"]].copy().sort_values("MONTH").reset_index(drop=True)
n_obs = len(dfm)

# Attach numeric exogenous cols
numeric_exo = []
for c in state_month.columns:
    if c in ("MONTH","y","YEAR"): continue
    if pd.api.types.is_numeric_dtype(state_month[c]):
        numeric_exo.append(c)
        dfm[c] = state_month.set_index("MONTH")[c].reindex(dfm["MONTH"]).values

# Calendar-only (always safe)
dfm["year"] = dfm["MONTH"].dt.year
dfm["month_num"] = dfm["MONTH"].dt.month
dfm["quarter"] = dfm["MONTH"].dt.quarter
dfm["month_sin"] = np.sin(2*np.pi*dfm["month_num"]/12.0)
dfm["month_cos"] = np.cos(2*np.pi*dfm["month_num"]/12.0)

# Choose lags/rolls adaptively
# heuristic: allow lags <= max_lag where max_lag <= max(1, n_obs//4)
max_lag = max(1, n_obs // 4)
cand_lags = [1,2,3,6,12]
LAGS = [l for l in cand_lags if l <= max_lag]

def add_lag_rolls(frame, lags):
    f = frame.copy()
    for lag in lags:
        f[f"lag_{lag}"] = f["y"].shift(lag)
    # rolling windows only if enough history
    def safe_roll(win):
        if n_obs >= win + 1:
            f[f"roll{win}_mean"] = f["y"].shift(1).rolling(win).mean()
            f[f"roll{win}_std"]  = f["y"].shift(1).rolling(win).std()
    for win in [3,6,12]:
        if win in lags or n_obs >= win + 3:
            safe_roll(win)
    return f

dfm_temp = add_lag_rolls(dfm, LAGS)

# Add exogenous lags if data is not tiny
EXO_LAGS = [1] if n_obs >= 6 else []
key_exo = [c for c in numeric_exo if any(k in c.lower() for k in
           ["storm_events_n","psps_events","out_","wildfire","pcpn","tavg","snow","snwd","incidents_n","doe417"])]
for c in key_exo:
    for lg in EXO_LAGS:
        dfm_temp[f"{c}_lag{lg}"] = dfm_temp[c].shift(lg)

# Build feature sets
BASE_FEATURES = ["year","month_num","quarter","month_sin","month_cos"] + [f"lag_{l}" for l in LAGS]
for w in [3,6,12]:
    if f"roll{w}_mean" in dfm_temp.columns: BASE_FEATURES.append(f"roll{w}_mean")
    if f"roll{w}_std"  in dfm_temp.columns: BASE_FEATURES.append(f"roll{w}_std")

EXO_FEATURES = [c for c in dfm_temp.columns if c not in BASE_FEATURES + ["MONTH","y","index"] and not c.endswith(("_x","_y"))]
# prefer exo lags over raw exo to reduce leakage risk
FEATURES_FULL = BASE_FEATURES + EXO_FEATURES
FEATURES_CAL = ["year","month_num","quarter","month_sin","month_cos"]  # always usable

# Drop rows with NaNs from lags/rolls (FULL) and separately keep CAL frame
df_full = dfm_temp.dropna().reset_index(drop=True)
df_cal  = dfm[["MONTH","y"] + FEATURES_CAL].dropna().reset_index(drop=True)

def time_split(df, feature_cols, test_frac=0.2):
    X = df[feature_cols].values
    y = df["y"].values
    dates = df["MONTH"].values
    if len(df) < 3:
        # if tiny, keep 1 sample for test if possible
        split_idx = max(1, len(df)-1)
    else:
        split_idx = max(1, min(len(df)-1, int(len(df)*(1-test_frac))))
    return (X[:split_idx], X[split_idx:], y[:split_idx], y[split_idx:], dates[:split_idx], dates[split_idx:])

def fit_eval(feature_frame, feature_cols, tag):
    X_train, X_test, y_train, y_test, dtr, dte = time_split(feature_frame, feature_cols)
    models = {
        "RandomForest": RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1),
        "KNN": KNeighborsRegressor(n_neighbors=5, weights="distance"),
        "SVM": SVR(kernel="rbf", C=10.0, epsilon=0.1, gamma="scale"),
        "MLP": MLPRegressor(hidden_layer_sizes=(128,64), activation="relu",
                            solver="adam", learning_rate_init=0.001,
                            max_iter=2000, random_state=42),
    }
    if HAS_XGB:
        models["XGBoost"] = XGBRegressor(
            n_estimators=800, max_depth=6, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
            random_state=42, tree_method="hist"
        )
    res, preds = [], {}
    scaler = StandardScaler()
    need_scale = lambda n: n in ["KNN","SVM","MLP"]

    # guard: empty test set → do a tiny holdout
    if len(X_test) == 0 and len(X_train) > 1:
        X_train, X_test = X_train[:-1], X_train[-1:].copy()
        y_train, y_test = y_train[:-1], y_train[-1:].copy()
        dtr, dte = dtr[:-1], dtr[-1:]

    for name, model in models.items():
        if need_scale(name):
            X_train_sc = scaler.fit_transform(X_train) if len(X_train) else X_train
            X_test_sc  = scaler.transform(X_test) if len(X_test) else X_test
            model.fit(X_train_sc, y_train)
            y_pred = model.predict(X_test_sc) if len(X_test_sc) else np.array([])
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test) if len(X_test) else np.array([])

        if len(y_test):
            rmse = float(np.sqrt(mean_squared_error(y_test, y_pred)))
            mae_ = float(mean_absolute_error(y_test, y_pred))
            r2_  = float(r2_score(y_test, y_pred))
            mape_ = mape(y_test, y_pred)
        else:
            rmse = mae_ = mape_ = np.nan; r2_ = np.nan

        res.append({"model": name, "set": tag, "RMSE": rmse, "MAE": mae_, "R2": r2_, "MAPE_%": mape_})
        preds[name] = (dte, y_pred, model, scaler if need_scale(name) else None)
    return pd.DataFrame(res), preds

results_all = []
preds_full = {}
preds_cal = {}

# Try FULL feature set first (if at least some rows after dropna)
if len(df_full) >= 3 and len(FEATURES_FULL) > 0:
    m_full, preds_full = fit_eval(df_full, FEATURES_FULL, tag="FULL")
    results_all.append(m_full)
else:
    print("ℹ️ FULL feature set not feasible (too few rows after lags/rolls or no features).")

# Always train CALENDAR-only as a fallback
m_cal, preds_cal = fit_eval(df_cal, FEATURES_CAL, tag="CAL")
results_all.append(m_cal)

metrics_df = pd.concat(results_all, ignore_index=True)
metrics_df = metrics_df.sort_values(["RMSE","set"]).reset_index(drop=True)
metrics_df.to_csv(OUT_DIR / "model_metrics_comparison.csv", index=False)
print("\n--- Model Comparison (sorted by RMSE) ---")
print(metrics_df.to_string(index=False))

# choose best among available sets
best_row = metrics_df.iloc[0]
best_set = best_row["set"]
best_model_name = best_row["model"]
pred_store = preds_full if best_set == "FULL" else preds_cal
dates_test, y_pred_best, fitted_model, fitted_scaler = pred_store[best_model_name]

# save per-model predictions
with pd.ExcelWriter(OUT_DIR / "model_predictions.xlsx") as xw:
    # truth from the same split as the best set
    frame = df_full if best_set == "FULL" else df_cal
    _, Xte, _, yte, _, dte = time_split(frame, FEATURES_FULL if best_set=="FULL" else FEATURES_CAL)
    pd.DataFrame({"DATE": dte, "y_true": yte}).to_excel(xw, sheet_name="truth", index=False)
    for name, (dte_i, yp_i, _, _) in pred_store.items():
        pd.DataFrame({"DATE": dte_i, f"{name}_pred": yp_i}).to_excel(xw, sheet_name=name, index=False)

# plots overlay
# build train/test for the chosen set again for plotting
frame = df_full if best_set == "FULL" else df_cal
feats = FEATURES_FULL if best_set=="FULL" else FEATURES_CAL
Xtr, Xte, ytr, yte, dtr, dte = time_split(frame, feats)

plt.figure()
plt.plot(dtr, ytr, marker="o", label="Train")
plt.plot(dte, yte, marker="o", label="Test Actual")
plt.plot(dte, y_pred_best, marker="o", linestyle="--", label=f"Test Pred • {best_model_name} ({best_set})")
plt.title("Statewide Monthly – Actual vs Predicted (Best Model)")
plt.xlabel("Month"); plt.ylabel(TARGET_COL.replace("_"," ").title()); plt.legend()
plt.tight_layout(); plt.savefig(OUT_DIR / "fig_actual_vs_pred_best.png", dpi=220); plt.close()

plt.figure()
plt.plot(dte, yte, marker="o", label="Test Actual")
for name, (dte_i, yp_i, _, _) in pred_store.items():
    plt.plot(dte_i, yp_i, marker="o", linestyle="--", label=f"{name} ({best_set})")
plt.title("Model Comparison – Test Predictions")
plt.xlabel("Month"); plt.ylabel(TARGET_COL.replace("_"," ").title()); plt.legend()
plt.tight_layout(); plt.savefig(OUT_DIR / "fig_model_comparison_overlay.png", dpi=220); plt.close()

# --------------------
# 6) 12-MONTH FORECAST (robust)
# --------------------
def _build_temporal_features(series_dates, series_vals, lags):
    tmp = pd.DataFrame({"MONTH": pd.to_datetime(series_dates), "y": series_vals}).sort_values("MONTH").reset_index(drop=True)
    tmp["year"] = tmp["MONTH"].dt.year
    tmp["month_num"] = tmp["MONTH"].dt.month
    tmp["quarter"] = tmp["MONTH"].dt.quarter
    tmp["month_sin"] = np.sin(2*np.pi*tmp["month_num"]/12.0)
    tmp["month_cos"] = np.cos(2*np.pi*tmp["month_num"]/12.0)
    for lag in lags:
        tmp[f"lag_{lag}"] = tmp["y"].shift(lag)
    def add_roll(win):
        if len(tmp) >= win + 1:
            tmp[f"roll{win}_mean"] = tmp["y"].shift(1).rolling(win).mean()
            tmp[f"roll{win}_std"]  = tmp["y"].shift(1).rolling(win).std()
    for w in [3,6,12]:
        add_roll(w)
    return tmp

# Temporal feature set mirrors BASE_FEATURES actually used
FORECAST_FEATURES = [f for f in FEATURES_CAL]  # always present
for l in LAGS:
    f = f"lag_{l}"
    if f in BASE_FEATURES: FORECAST_FEATURES.append(f)
for w in [3,6,12]:
    for suffix in ["mean","std"]:
        f = f"roll{w}_{suffix}"
        if f in BASE_FEATURES: FORECAST_FEATURES.append(f)

# Train temporal-only clone of winning algo on ALL history of the chosen set's target
model_best_fore = clone(fitted_model)
history = df_full if best_set == "FULL" else df_cal
series_dates = list(history["MONTH"].values)
series_vals  = list(history["y"].values)

X_temporal_frame = _build_temporal_features(series_dates, series_vals, LAGS)
X_temporal = X_temporal_frame.dropna()[FORECAST_FEATURES].values
y_temporal = X_temporal_frame.dropna()["y"].values

use_scaler_fore = isinstance(fitted_scaler, StandardScaler)
if use_scaler_fore:
    scaler_fore = StandardScaler().fit(X_temporal)
    X_temporal_sc = scaler_fore.transform(X_temporal)
    model_best_fore.fit(X_temporal_sc, y_temporal)
else:
    model_best_fore.fit(X_temporal, y_temporal)

last_month = pd.to_datetime(max(series_dates))
future_months = pd.date_range(last_month + pd.offsets.MonthBegin(1), periods=12, freq="MS")

future_preds = []
for fm in future_months:
    series_dates.append(fm)
    series_vals.append(series_vals[-1] if len(series_vals) else 0.0)
    tmp = _build_temporal_features(series_dates, series_vals, LAGS)
    row = tmp.loc[tmp["MONTH"] == pd.to_datetime(fm)].tail(1)
    # ensure all needed columns exist (may be missing for very short history)
    for col in FORECAST_FEATURES:
        if col not in row.columns: row[col] = np.nan
    row = row[FORECAST_FEATURES].fillna(method="ffill", axis=0).fillna(0.0)
    X_row = row.values
    if use_scaler_fore:
        X_row = scaler_fore.transform(X_row)
    y_hat = float(model_best_fore.predict(X_row)[0])
    future_preds.append(y_hat)
    series_vals[-1] = y_hat

# If history was extremely short and model fit failed to produce, fallback naive
if not future_preds or any(np.isnan(future_preds)):
    print("ℹ️ Falling back to naive forecast due to short history.")
    # seasonal mean by calendar month if >= 12 obs, else rolling mean of last k
    base = state_month.copy()
    if len(base) >= 12:
        base["m"] = base["MONTH"].dt.month
        month_avg = base.groupby("m")["y"].mean().to_dict()
        future_preds = [float(month_avg.get(m, base["y"].iloc[-1])) for m in [d.month for d in future_months]]
    else:
        k = min(3, max(1, len(base)))
        last_mean = float(base["y"].tail(k).mean())
        future_preds = [last_mean for _ in range(12)]

forecast_df = pd.DataFrame({"MONTH": future_months, "Forecast_y": future_preds})
forecast_df.to_csv(OUT_DIR / "statewide_12mo_forecast_best_model.csv", index=False)

plt.figure()
plt.plot(state_month["MONTH"], state_month["y"], marker="o", label="History")
plt.plot(forecast_df["MONTH"], forecast_df["Forecast_y"], marker="o", linestyle="--",
         label=f"Forecast ({best_model_name} {best_set if best_set else ''} / temporal)")
plt.title("12-Month Forecast – Statewide")
plt.xlabel("Month"); plt.ylabel(TARGET_COL.replace("_"," ").title()); plt.legend()
plt.tight_layout(); plt.savefig(OUT_DIR / "fig_12mo_forecast_best.png", dpi=220); plt.close()

# --------------------
# 7) SUMMARY
# --------------------
def fmt0(x):
    try: return f"{x:,.0f}"
    except: return str(x)

metrics_df.to_csv(OUT_DIR / "model_metrics_comparison.csv", index=False)
lines = []
lines.append("# EAGLE-I Aggregated (Statewide) — Summary")
lines.append("")
lines.append("## Inputs")
lines.append(f"- File: {INPUT_CSV}")
lines.append(f"- Target: {TARGET_COL}")
lines.append("")
lines.append("## Tables")
lines.append("- state_month.csv, state_year.csv, eaglei_state_trends.xlsx")
if (OUT_DIR / "state_month_with_exogenous.csv").exists():
    lines.append("- state_month_with_exogenous.csv")
lines.append("")
lines.append("## Figures")
lines.append("- fig_state_month.png, fig_state_year.png")
lines.append("- fig_actual_vs_pred_best.png, fig_model_comparison_overlay.png, fig_12mo_forecast_best.png")
lines.append("")
lines.append("## Model Comparison (sorted by RMSE)")
lines.append(metrics_df.to_string(index=False))
lines.append("")
lines.append("## Best Model")
lines.append(f"- {best_model_name} on {best_set} (RMSE={fmt0(best_row['RMSE'])}, MAE={fmt0(best_row['MAE'])}, R²={best_row['R2']:.3f}, MAPE≈{best_row['MAPE_%']:.2f}%)")
(OUT_DIR / "SUMMARY.txt").write_text("\n".join(lines), encoding="utf-8")

print("\n✅ All done. See outputs in:", OUT_DIR.resolve())


✅ Loaded: /content/Eaglei_CA_Outages_Agg.csv
Columns: ['state', 'year', 'month', 'outage_count', 'max_outage_duration', 'customer_weighted_hours']
⚠️ Dropping 10 rows with invalid month outside 1..12.
✅ Exported summary tables to /content/eaglei_agg_outputs_state
✅ Added exogenous features: incidents_n, out_customers_impacted_sum, storm_crop_damage_usd, storm_events_n, storm_prop_damage_usd, wildfire_incidents_n
ℹ️ FULL feature set not feasible (too few rows after lags/rolls or no features).


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.




--- Model Comparison (sorted by RMSE) ---
       model set         RMSE          MAE        R2    MAPE_%
     XGBoost CAL 7.072923e+06 4.429189e+06 -0.098735 58.385910
         KNN CAL 7.164929e+06 4.479373e+06 -0.127506 57.867605
RandomForest CAL 7.830953e+06 5.208296e+06 -0.346865 76.110759
         SVM CAL 8.179285e+06 4.629845e+06 -0.469351 41.357787
         MLP CAL 9.829752e+06 7.131166e+06 -1.122170 88.502543


/tmp/ipython-input-3087929840.py:617: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipython-input-3087929840.py:617: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipython-input-3087929840.py:617: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipython-input-3087929840.py:617: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipython-input-3087929840.py:617: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipython-input-3087929840.py:617: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a f


✅ All done. See outputs in: /content/eaglei_agg_outputs_state


In [58]:
import os

output_dir = "/content/eaglei_agg_outputs" # Updated output directory to the user's request

print("Output files generated in:", output_dir)
for file in os.listdir(output_dir):
    print(f"- {os.path.join(output_dir, file)}")

Output files generated in: /content/eaglei_agg_outputs
- /content/eaglei_agg_outputs/county_month.csv
- /content/eaglei_agg_outputs/state_year.csv
- /content/eaglei_agg_outputs/fig_county_year_matrix.png
- /content/eaglei_agg_outputs/fig_state_year_customers_out.png
- /content/eaglei_agg_outputs/fig_state_month_customers_out.png
- /content/eaglei_agg_outputs/top10_counties_total_customers_out.csv
- /content/eaglei_agg_outputs/fig_top10_counties.png
- /content/eaglei_agg_outputs/state_month.csv
- /content/eaglei_agg_outputs/county_year.csv
- /content/eaglei_agg_outputs/county_year_matrix_customers_out.csv
- /content/eaglei_agg_outputs/eaglei_agg_trends.xlsx



# **EAGLE-I Statewide Outputs and Forecasts**

## **Description**
This directory contains **state-level analytical outputs** generated from the **EAGLE-I aggregated dataset**.  
It focuses on monthly and yearly statewide trends, model evaluations, forecasts, and performance diagnostics.  
These files support **time-series**, **regression**, and **forecasting analyses** for California’s statewide outage data.

---

## **Contents**

| **File** | **Description** |
|-----------|-----------------|
| `state_month.csv` | Aggregated monthly statewide outage metrics. |
| `state_year.csv` | Aggregated yearly statewide outage metrics. |
| `eaglei_state_trends.xlsx` | Combined workbook with monthly and yearly trend sheets. |
| `state_month_with_exogenous.csv` | Statewide data merged with exogenous variables (weather, storm events, wildfires, etc.). |
| `model_metrics_comparison.csv` | Model comparison results (RMSE, MAE, R², MAPE). |
| `model_predictions.xlsx` | Model-specific prediction outputs for test sets. |
| `SUMMARY.txt` | Plain-text summary of the modeling pipeline, results, and best model. |
| `statewide_forecast_through_2026.csv` | Forecast of statewide outage intensity through December 2026. |
| **Figures (.png)** | Visualization files for trends, forecasts, and model comparisons. |

---

## **Use Cases**
- Evaluate statewide outage patterns over time.  
- Compare ML models (Random Forest, SVM, XGBoost, etc.).  
- Generate and visualize multi-year forecasts (to 2026).  
- Integrate results into reports or dashboards summarizing energy system reliability.  


In [48]:
import pandas as pd
import plotly.express as px
import os

output_dir = "/content/eaglei_agg_outputs_state"

# Display state-level monthly trends
state_month_df = pd.read_csv(os.path.join(output_dir, "state_month.csv"))
print("Statewide Monthly Trends:")
fig_month = px.line(state_month_df, x='MONTH', y='y', title='Statewide Monthly Trends')
fig_month.show()

# Display state-level yearly trends
state_year_df = pd.read_csv(os.path.join(output_dir, "state_year.csv"))
print("\nStatewide Yearly Trends:")
fig_year = px.line(state_year_df, x='YEAR', y='y', title='Statewide Yearly Trends')
fig_year.show()


# Display model comparison metrics
model_metrics_df = pd.read_csv(os.path.join(output_dir, "model_metrics_comparison.csv"))
print("\nModel Comparison Metrics:")
display(model_metrics_df)

# Display forecast data
forecast_df = pd.read_csv(os.path.join(output_dir, "statewide_forecast_through_2026.csv"))
print("\nForecast to 2026:")
fig_forecast = px.line(forecast_df, x='MONTH', y='Forecast_y', title='Forecast to 2026')
fig_forecast.show()

# Optional: Combine historical and forecast for a single plot
# Reload state_month_df as it was used in the previous plot
state_month_df_hist = pd.read_csv(os.path.join(output_dir, "state_month.csv"))
combined_df = pd.concat([state_month_df_hist.rename(columns={'y': 'Value'}),
                         forecast_df.rename(columns={'Forecast_y': 'Value'})], ignore_index=True)
combined_df['Type'] = ['History'] * len(state_month_df_hist) + ['Forecast'] * len(forecast_df)

print("\nHistorical and Forecast Combined:")
fig_combined = px.line(combined_df, x='MONTH', y='Value', color='Type', title='Statewide Monthly History and Forecast to 2026')
fig_combined.show()

# List generated figures (matplotlib - still exist unless deleted)
print("\nGenerated Matplotlib Figures (paths):")
for file in os.listdir(output_dir):
    if file.endswith(".png"):
        print(f"- {os.path.join(output_dir, file)}")

Statewide Monthly Trends:



Statewide Yearly Trends:



Model Comparison Metrics:


,model,set,RMSE,MAE,R2,MAPE_%
0,XGBoost,CAL,7.072923e+06,4.429189e+06,-0.098735,58.385910
1,KNN,CAL,7.164929e+06,4.479373e+06,-0.127506,57.867605
2,RandomForest,CAL,7.830953e+06,5.208296e+06,-0.346865,76.110759
3,SVM,CAL,8.179285e+06,4.629845e+06,-0.469351,41.357787
4,MLP,CAL,9.829752e+06,7.131166e+06,-1.122170,88.502543



Forecast to 2026:



Historical and Forecast Combined:



Generated Matplotlib Figures (paths):
- /content/eaglei_agg_outputs_state/fig_12mo_forecast_best.png
- /content/eaglei_agg_outputs_state/usa_month_customers_out.png
- /content/eaglei_agg_outputs_state/fig_actual_vs_pred_best.png
- /content/eaglei_agg_outputs_state/fig_state_year.png
- /content/eaglei_agg_outputs_state/state_month_customers_out.png
- /content/eaglei_agg_outputs_state/fig_state_month.png
- /content/eaglei_agg_outputs_state/fig_forecast_through_2026.png
- /content/eaglei_agg_outputs_state/fig_model_comparison_overlay.png


# **EAGLE-I Aggregated Trends (State + County Analysis)**

## **Description**
This directory contains **aggregated outage trend data** at both the **statewide** and **county** levels.  
It provides historical and comparative datasets to analyze **spatial**, **temporal**, and **sectoral** outage distributions across California.  
The folder supports **correlation**, **ranking**, and **cross-scale comparison analyses**.

---

## **Contents**

| **File** | **Description** |
|-----------|-----------------|
| `state_trends.csv` | State-level monthly and yearly outage totals. |
| `county_trends.csv` | County-level monthly outage data. |
| `county_year_matrix.csv` | Pivot table showing yearly outage values by county. |
| `top10_counties.csv` | Ranked list of counties with the highest average or total outages. |
| `aggregated_summary.xlsx` | Combined workbook with state, county, and matrix summaries. |
| **Figures (.png)** | Charts of top-10 counties, spatial distributions, and comparative trends. |

---

## **Use Cases**
- Identify top outage-impacted counties.  
- Compare local (county) and statewide trends.  
- Build visual or GIS-based dashboards.  
- Support reliability, equity, and infrastructure resilience analyses.  


In [59]:
import pandas as pd
import plotly.express as px
import os

output_dir = "/content/eaglei_agg_outputs"

# Display state-level monthly trends
state_month_df = pd.read_csv(os.path.join(output_dir, "state_month.csv"))
print("Statewide Monthly Trends:")
fig_state_month = px.line(state_month_df, x='MONTH', y='total_customers_out', title='Statewide Monthly Total Customers Out')
fig_state_month.show()

# Display state-level yearly trends
state_year_df = pd.read_csv(os.path.join(output_dir, "state_year.csv"))
print("\nStatewide Yearly Trends:")
fig_state_year = px.line(state_year_df, x='YEAR', y='total_customers_out', title='Statewide Yearly Total Customers Out')
fig_state_year.show()

# Display top 10 counties by cumulative customers out
top10_counties_df = pd.read_csv(os.path.join(output_dir, "top10_counties_total_customers_out.csv"))
print("\nTop 10 Counties by Cumulative Customers Out:")
fig_top10 = px.bar(top10_counties_df, x='county', y='total_customers_out', title='Top 10 Counties by Cumulative Customers Out')
fig_top10.show()

# Display county-year matrix (heatmap-like)
county_year_matrix_df = pd.read_csv(os.path.join(output_dir, "county_year_matrix_customers_out.csv"), index_col=0)
print("\nCounty-Year Matrix (Customers Out):")
fig_matrix = px.imshow(county_year_matrix_df,
                        labels=dict(x="Year", y="County", color="Total Customers Out"),
                        x=county_year_matrix_df.columns,
                        y=county_year_matrix_df.index,
                        title="County x Year Matrix – Total Customers Out")
fig_matrix.show()

# List generated figures (matplotlib - from the original script)
print("\nGenerated Matplotlib Figures (paths from the original script):")
output_state_dir = "/content/eaglei_agg_outputs_state"
if os.path.exists(output_state_dir):
    for file in os.listdir(output_state_dir):
        if file.endswith(".png"):
            print(f"- {os.path.join(output_state_dir, file)}")

print("\nGenerated Matplotlib Figures (paths from the trend analysis script):")
if os.path.exists(output_dir):
    for file in os.listdir(output_dir):
        if file.endswith(".png"):
            print(f"- {os.path.join(output_dir, file)}")

Statewide Monthly Trends:



Statewide Yearly Trends:



Top 10 Counties by Cumulative Customers Out:



County-Year Matrix (Customers Out):



Generated Matplotlib Figures (paths from the original script):
- /content/eaglei_agg_outputs_state/fig_12mo_forecast_best.png
- /content/eaglei_agg_outputs_state/fig_actual_vs_pred_best.png
- /content/eaglei_agg_outputs_state/fig_state_year.png
- /content/eaglei_agg_outputs_state/fig_state_month.png
- /content/eaglei_agg_outputs_state/fig_forecast_through_2026.png
- /content/eaglei_agg_outputs_state/fig_model_comparison_overlay.png

Generated Matplotlib Figures (paths from the trend analysis script):
- /content/eaglei_agg_outputs/fig_county_year_matrix.png
- /content/eaglei_agg_outputs/fig_state_year_customers_out.png
- /content/eaglei_agg_outputs/fig_state_month_customers_out.png
- /content/eaglei_agg_outputs/fig_top10_counties.png
